In [7]:
# select names contain most productive author
import sys
import io
import os

Dataset = "pubmed"

filePath = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(filePath)

LabeledRecords_original = []

for file in listfiles:
    if not file.startswith('.'):
        with open(filePath+file, 'r', encoding = 'utf8') as f:
            for line in f:
                read_data = line.split("\t")
                # get ride of bad formated lines
                if(len(read_data)==13):
                    paper_detail = {"paperID": read_data[0], "authorID":read_data[1], "belone file": file}
                    LabeledRecords_original.append(paper_detail)
        f.close()
print("Total labeled records:",len(LabeledRecords_original), "records")
print(LabeledRecords_original[:3])

Total labeled records: 140266 records
[{'paperID': '27406695', 'authorID': '0000-0002-9697-0962', 'belone file': 'canopy_j_read'}, {'paperID': '26876744', 'authorID': '0000-0002-9697-0962', 'belone file': 'canopy_j_read'}, {'paperID': '26978780', 'authorID': '0000-0002-9697-0962', 'belone file': 'canopy_j_read'}]


In [8]:
import random
from collections import Counter

# pick top productive author
all_author_aid = [d['authorID'] for d in LabeledRecords_original]
author_paper_count = Counter(all_author_aid)
top_20_author = author_paper_count.most_common(20)
print(top_20_author)
top_20_aid = [author[0] for author in top_20_author]

# find names (files) contains those productive author
top_author_belone_file = list(set([file["belone file"] for file in LabeledRecords_original if file["authorID"] in top_20_aid]))

# for each of 20 names, form one vs rest classification
for top_author_file_name in top_author_belone_file:
    ingroup_papers = [pid for pid in LabeledRecords_original if pid["belone file"]==top_author_file_name]
    all_ingroup_pids = [paper["paperID"] for paper in ingroup_papers]
    print(top_author_file_name,":",len(all_ingroup_pids))
    
    # find most productive author id
    positive_aid = list(set([paper["authorID"] for paper in ingroup_papers if paper["authorID"] in top_20_aid]))
    positive_pids = [paper["paperID"] for paper in ingroup_papers if paper["authorID"] in top_20_aid]
    print("C1: ", positive_aid, ":", len(positive_pids))
    
    # rest of papers as negative class
    negative_pids = [pid for pid in all_ingroup_pids if pid not in positive_pids]
    print("C2: ",len(negative_pids))
    
    # random len(positive class) from negative class
    final_negative_pids = random.sample(negative_pids, min(len(positive_pids), len(negative_pids)))
    
    print("Final class condition, C1:", len(positive_pids), " C2:", len(final_negative_pids))
    
    fileSaveDir = "top_author_vs_rest(same_name)/"
    if not os.path.exists(fileSaveDir):
        os.makedirs(fileSaveDir)
    
    newf = open("top_author_vs_rest(same_name)/"+top_author_file_name+".txt", "w",encoding='utf8')
    # write positive class
    for pids in positive_pids:
        newf.write(pids+"\t"+ "0" + "\n")
    # write negative class
    for pids in final_negative_pids:
        newf.write(pids+"\t"+ "1"+ "\n")
    newf.close()


[('0000-0002-2381-2349', 587), ('0000-0002-9955-6003', 487), ('0000-0002-4295-6129', 423), ('0000-0003-4876-8839', 356), ('0000-0001-5205-8245', 343), ('0000-0001-7108-9776', 331), ('0000-0001-6505-4139', 322), ('0000-0003-2950-3632', 279), ('0000-0002-1619-8328', 268), ('0000-0001-8532-2727', 267), ('0000-0002-9152-6512', 265), ('0000-0001-5664-9392', 252), ('0000-0003-3894-5301', 251), ('0000-0002-5555-9034', 244), ('0000-0002-3765-3318', 238), ('0000-0003-3709-1690', 237), ('0000-0002-3101-1138', 232), ('0000-0003-0960-6415', 231), ('0000-0002-3894-4811', 222), ('0000-0002-1940-6428', 219)]
canopy_x_li : 867
C1:  ['0000-0002-5555-9034'] : 244
C2:  623
Final class condition, C1: 244  C2: 244
canopy_c_nogueira : 303
C1:  ['0000-0003-2950-3632'] : 279
C2:  24
Final class condition, C1: 279  C2: 24
canopy_m_wu : 658
C1:  ['0000-0002-1940-6428'] : 219
C2:  439
Final class condition, C1: 219  C2: 219
canopy_r_ross : 374
C1:  ['0000-0003-4876-8839'] : 356
C2:  18
Final class condition, C1: